In [1]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import normalize

#### Participant simulation

First we'll create *n* participants in our experiment.

In [2]:
n = 1000

For each of these participants, we'll randomly assign them some (Compliance, Response) behavior combination.

In [30]:
compliance_partitions = ['always_taker', 
                         'complier', 
                         'defier', 
                         'never_taker']
response_partitions = ['always_better', 
                       'helped', 
                       'hurt', 
                       'never_better']

# Take the cross product of these sets of types

partition_types = product(compliance_partitions, response_partitions)
partition_types = np.array(list(partition_types))


In [4]:
# We'll simulate probabilities that our participants 
# will belong to one of the 16 possible behavior combinations

random_floats = np.random.random([1,16]) 
partition_probabilities = normalize(random_floats, 'l1')
partition_probabilities = partition_probabilities.flatten()

# to be a true set of probabilities, the vector sum 
# needs to be 1

# sometimes this can fail because of precision errors, 
# so let's assert it

assert partition_probabilities.sum() == 1 

In [5]:
# drawing participant compliance and response behaviors according to the
# specified distribution

participant_partition = np.random.choice(
                            range(len(partition_types)), 
                            n, 
                            p=partition_probabilities
                        )

compliance_type, response_type = list(zip(*partition_types[participant_partition]))

# assigning participants to Control and Treatment groups 
# with 50% probability

assignments = ['control', 'treatment']
participant_assignment = np.random.choice(assignments, n)

# compiling all information into our dataframe

df = pd.DataFrame({'assignment': participant_assignment,
                   'compliance_type': compliance_type,
                   'response_type': response_type})

### Simulate whether participants took treatment

Depending on assignment and compliance type, we can simulate whether or not each participant took the treatment.

In [31]:
# if the participant is an always_taker, 
# they'll always take the treatment.

df['took_treatment'] = (df.compliance_type == 'always_taker')

# if they're a complier, they'll take the treatment 
# as long as they're in the treatment condition.

df['took_treatment'] = df['took_treatment'] \
                        | ( 
                            (df.compliance_type == 'complier') 
                            & 
                            (df.assignment == 'treatment') 
                          )

# if they're a defier, they'll only take the treatment 
# if they were in the control condition.

df['took_treatment'] = df['took_treatment'] \
                        | ( 
                            (df.compliance_type == 'defier') 
                            & 
                            (df.assignment == 'control') 
                          )

### Simulate Outcomes

Now we can simulate outcomes from the experiment.

Depending on whether they took the treatment and their `response_type`, did they end up in a Good or Bad state after the experiment's conclusion?

In [7]:
# if the participant is of the always_better type, 
# they'll definitely have a good outcome.

df['good_outcome'] = (df.response_type == 'always_better') 

# if the participant is of the 'helped' type, 
# they'll have a good outcome as long as they
# took treatment.

df['good_outcome'] = df['good_outcome'] \
                     | ( 
                        (df.response_type == 'helped') 
                        & 
                        (df.took_treatment) 
                       )

# Otherwise, the outcome is going to be bad
# and the column will have a False value.

In [8]:
# we can observe the probabilities of each 
# (Treatment, Outcome) combination
# that would emerge, given the assignment

df['n'] = 1
results = df.groupby(['assignment', 
                      'took_treatment', 
                      'good_outcome']).count().n
results = results.to_frame()
results['assignment_n'] = results.groupby('assignment').transform('sum')

In [9]:
p_states = results.n / results.assignment_n
p_states = p_states.rename("P( X, Y | Z )")

# display

p_states.to_frame()

P( X, Y | Z )
assignment took_treatment good_outcome               
control    False          False              0.362173
                          True               0.144869
           True           False              0.259557
                          True               0.233400
treatment  False          False              0.411531
                          True               0.141153
           True           False              0.190855
                          True               0.256461

In [11]:
# intent-to-treat average treatment effect

# sum up all 'good' from the Treatment column, and subtract by all 'good' from the Control


In [12]:
# Intent to treat analysis:

itt_ate = df[df.assignment == 'treatment'].good_outcome.mean() \
          - df[df.assignment == 'control'].good_outcome.mean() 

print("ATE: %6.4f" % itt_ate)

ATE: 0.0193


In [13]:
# Causal analysis

# get all the probabilities we need: p(z,x,y) for each z,x,y combination
states = list(product(['treatment','control'],[0,1], [0,1]))

p_states = {f"{assignment}/{'treated' if took == 1 else 'untreated'}/{'good' if outcome ==1 else 'bad'}" : 
                ( (df[df.assignment == assignment].took_treatment == took)
                   & (df[df.assignment == assignment].good_outcome == outcome)  ).mean()
                for assignment, took, outcome in states
                }

In [14]:
p_states

{'treatment/untreated/bad': 0.4115308151093439,
 'treatment/untreated/good': 0.1411530815109344,
 'treatment/treated/bad': 0.1908548707753479,
 'treatment/treated/good': 0.25646123260437376,
 'control/untreated/bad': 0.36217303822937624,
 'control/untreated/good': 0.1448692152917505,
 'control/treated/bad': 0.2595573440643863,
 'control/treated/good': 0.23340040241448692}

In [16]:
true_ate = (df.response_type == 'helped').mean() - (df.response_type == 'hurt').mean()
true_ate

0.16300000000000003

In [17]:
# linear programming problem
import pulp
min_problem = pulp.LpProblem("min ATE", pulp.LpMinimize)
max_problem = pulp.LpProblem("max ATE", pulp.LpMaximize)

# our hidden variables are the the probability of being in one of the partitions
partition_names = ['/'.join([compliance, response]) for compliance, response in partition_types]
q = {partition: pulp.LpVariable(partition, lowBound=0) for partition in partition_names}

In [18]:
# since our hidden vars are probabilities the sum of them should all be under 1
min_problem += sum([v for k,v in q.items()]) == 1

In [19]:
# statements
p_treatment_untreated_bad = q['never_taker/never_better'] + q['defier/never_better'] \
                             + q['never_taker/helped'] + q['defier/helped']

p_treatment_untreated_good = q['never_taker/always_better'] + q['defier/always_better'] \
                             + q['never_taker/hurt'] + q['defier/hurt']

p_treatment_treated_bad = q['always_taker/never_better'] + q['complier/never_better'] \
                             + q['always_taker/hurt'] + q['complier/hurt']

p_treatment_treated_good = q['always_taker/always_better'] + q['complier/always_better'] \
                             + q['always_taker/helped'] + q['complier/helped']

p_control_untreated_bad = q['never_taker/never_better'] + q['complier/never_better'] \
                             + q['never_taker/helped'] + q['complier/helped']

p_control_untreated_good = q['never_taker/always_better'] + q['complier/never_better'] \
                             + q['never_taker/hurt'] + q['complier/hurt']

p_control_treated_bad = q['always_taker/never_better'] + q['defier/never_better'] \
                             + q['always_taker/hurt'] + q['defier/hurt']

p_control_treated_good = q['always_taker/always_better'] + q['defier/always_better'] \
                             + q['always_taker/helped'] + q['defier/helped']

In [20]:
# there's a natural mapping from probabilities we see to the hidden variables we have
# we'll spell these out one by one.
# there are probably smarter ways to express this as a vector operation,
# but this is easier to understand
min_problem += p_treatment_untreated_bad == p_states['treatment/untreated/bad']

min_problem += p_treatment_untreated_good == p_states['treatment/untreated/good']

min_problem += p_treatment_treated_bad == p_states['treatment/treated/bad']

min_problem += p_control_untreated_bad == p_states['control/untreated/bad']

min_problem += p_control_untreated_good == p_states['control/untreated/good']

min_problem += p_control_treated_bad == p_states['control/treated/bad']

#min_problem += p_treatment_treated_good == p_states['treatment/treated/good']

#min_problem += p_control_treated_good == p_states['control/treated/good']

In [21]:
min_problem += q['complier/helped'] + q['defier/helped'] + q['always_taker/helped'] + q['never_taker/helped'] \
              - q['complier/hurt'] - q['defier/hurt'] - q['always_taker/hurt'] - q['never_taker/hurt']

In [22]:
pulp.LpStatus[min_problem.solve()]

'Optimal'

In [23]:
q_min = {partition:pulp.value(partition_p) for partition, partition_p in q.items()}
q_min

{'always_taker/always_better': 0.23711654,
 'always_taker/helped': 0.0,
 'always_taker/hurt': 0.18713874,
 'always_taker/never_better': 0.0,
 'complier/always_better': 0.0,
 'complier/helped': 0.019344696,
 'complier/hurt': 0.0,
 'complier/never_better': 0.0037161338,
 'defier/always_better': 0.0,
 'defier/helped': 0.0,
 'defier/hurt': 0.0,
 'defier/never_better': 0.072418607,
 'never_taker/always_better': 0.0,
 'never_taker/helped': 0.0,
 'never_taker/hurt': 0.14115308,
 'never_taker/never_better': 0.33911221}

In [26]:
def apply_constraints(problem):
    
    # our hidden variables are the the probability of being in one of the partitions
    partition_names = ['/'.join([compliance, response]) for compliance, response in partition_types]
    q = {partition: pulp.LpVariable(partition, lowBound=0) for partition in partition_names}
    
    problem += sum([v for k,v in q.items()]) == 1
    
    p_treatment_untreated_bad = q['never_taker/never_better'] + q['defier/never_better'] \
                             + q['never_taker/helped'] + q['defier/helped']

    p_treatment_untreated_good = q['never_taker/always_better'] + q['defier/always_better'] \
                                 + q['never_taker/hurt'] + q['defier/hurt']

    p_treatment_treated_bad = q['always_taker/never_better'] + q['complier/never_better'] \
                                 + q['always_taker/hurt'] + q['complier/hurt']

    p_treatment_treated_good = q['always_taker/always_better'] + q['complier/always_better'] \
                                 + q['always_taker/helped'] + q['complier/helped']

    p_control_untreated_bad = q['never_taker/never_better'] + q['complier/never_better'] \
                                 + q['never_taker/helped'] + q['complier/helped']

    p_control_untreated_good = q['never_taker/always_better'] + q['complier/never_better'] \
                                 + q['never_taker/hurt'] + q['complier/hurt']

    p_control_treated_bad = q['always_taker/never_better'] + q['defier/never_better'] \
                                 + q['always_taker/hurt'] + q['defier/hurt']

    p_control_treated_good = q['always_taker/always_better'] + q['defier/always_better'] \
                                 + q['always_taker/helped'] + q['defier/helped']
    
    problem += p_treatment_untreated_bad == p_states['treatment/untreated/bad']
    problem += p_treatment_untreated_good == p_states['treatment/untreated/good']
    problem += p_treatment_treated_bad == p_states['treatment/treated/bad']
    problem += p_control_untreated_bad == p_states['control/untreated/bad']
    problem += p_control_untreated_good == p_states['control/untreated/good']
    problem += p_control_treated_bad == p_states['control/treated/bad']
    
    problem += q['complier/helped'] + q['defier/helped'] + q['always_taker/helped'] + q['never_taker/helped'] \
              - q['complier/hurt'] - q['defier/hurt'] - q['always_taker/hurt'] - q['never_taker/hurt']
    
    
    status = pulp.LpStatus[problem.solve()]
    
    if status != 'Optimal':
        raise ValueError('Infeasible')
        
    q_solved = {partition:pulp.value(partition_p) for partition, partition_p in q.items()}
    
    return q_solved


In [27]:
q_min = apply_constraints(min_problem)
q_max = apply_constraints(max_problem)

PulpSolverError: Pulp: Error while executing /usr/local/lib/python3.6/site-packages/pulp/solverdir/cbc/osx/64/cbc

In [28]:
ate = lambda q: q['complier/helped'] + q['defier/helped'] + q['always_taker/helped'] + q['never_taker/helped'] \
              - q['complier/hurt'] - q['defier/hurt'] - q['always_taker/hurt'] - q['never_taker/hurt']

min_ate = ate(q_min)

In [29]:
max_ate = ate(q_max)

NameError: name 'q_max' is not defined

In [ ]:
min_ate

In [ ]:
max_ate